<a href="https://colab.research.google.com/github/ysc4/CCADMACL_EXERCISES_COM222-ML/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [172]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [173]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [174]:
housing_features = pd.DataFrame(housing.data, columns=housing.feature_names)
housing_target = pd.DataFrame(housing.target, columns=['MedHouseVal'])

In [175]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


1. Use any filter method to select the best features

In [177]:
housing_features.size

165120

In [178]:
housing_target.shape

(20640, 1)

In [179]:
# put your answer here

from sklearn.feature_selection import f_classif

threshold = 5
high_score_features = []
feature_scores = f_classif(housing_features, housing_target.values.ravel())[0]

In [180]:
for score, f_name in sorted(zip(feature_scores, housing_features.columns), reverse=True)[:threshold]:
  high_score_features.append(f_name)

housing_fc = housing_features[high_score_features]
housing_fc.columns

Index(['MedInc', 'Latitude', 'Population', 'HouseAge', 'Longitude'], dtype='object')

2. Use any wrapper method to select the best features

In [181]:
# put your answer here

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
selector = RFE(model_rf, n_features_to_select=5,step=1)

selector = selector.fit(housing_features, housing_target.values.ravel())
selector_ind = selector.get_support()
housing_rfe = housing_features.iloc[:, selector_ind]
housing_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

3. Use any embedded methood to select the best features

In [182]:
from sklearn.model_selection import train_test_split

X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.DataFrame(housing.target, columns=['MedHouseVal'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [183]:
# put your answer here

from sklearn.feature_selection import SelectFromModel

model_sfm = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
model_sfm.fit(X_train, y_train.values.ravel())
y_pred_sfm = model_sfm.predict(X_test)

sel_sfm = SelectFromModel(model_sfm, prefit=True)
sel_sfm_index = sel_sfm.get_support()
housing_sfm = housing_features.iloc[:, sel_sfm_index]
housing_sfm.columns

Index(['MedInc', 'AveOccup'], dtype='object')

# Model Training

In [184]:
from sklearn.model_selection import train_test_split

housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

Default Model

In [185]:
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.DataFrame(housing.target, columns=['MedHouseVal'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [186]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
model_rf = model_rf.fit(X_train, y_train.values.ravel())
y_pred_rf = model_rf.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

Model with filtered features

In [187]:
X_train, X_test, y_train, y_test = train_test_split(housing_fc, housing_target, test_size=0.2, random_state=42)
model_fc = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
model_fc = model_fc.fit(X_train, y_train.values.ravel())
y_pred_fc = model_fc.predict(X_test)

rmse_fc = np.sqrt(mean_squared_error(y_test, y_pred_fc))

Model with wrapper method

In [188]:
X_train, X_test, y_train, y_test = train_test_split(housing_rfe, housing_target, test_size=0.2, random_state=42)
model_sfe = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
model_sfe = model_sfe.fit(X_train, y_train.values.ravel())
y_pred_sfe = model_sfe.predict(X_test)

rmse_sfe = np.sqrt(mean_squared_error(y_test, y_pred_sfe))

Model with embedded method


In [191]:
X_train, X_test, y_train, y_test = train_test_split(housing_sfm, housing_target, test_size=0.2, random_state=42)
model_sfm = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=3)
model_sfm = model_sfm.fit(X_train, y_train.values.ravel())
y_pred_sfm = model_sfm.predict(X_test)

rmse_sfm = np.sqrt(mean_squared_error(y_test, y_pred_sfm))

# Calculated RMSEs

In [192]:
print(f'Default RMSE: {rmse_rf}')
print(f'Model with filtered method RMSE: {rmse_fc}')
print(f'Model with wrapper method RMSE: {rmse_sfe}')
print(f'Model with embedded method RMSE: {rmse_sfm}')


Default RMSE: 0.7750447841432082
Model with filtered method RMSE: 0.8085275568858331
Model with wrapper method RMSE: 0.7750566258629724
Model with embedded method RMSE: 0.7823934576498581
